<a href="https://colab.research.google.com/github/gvxfnv/lizavavilova/blob/main/%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D0%B06.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import sqlite3
import os
conn = sqlite3.connect('library.db')
cursor = conn.cursor()
create_table_query = '''
CREATE TABLE IF NOT EXISTS books (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    title TEXT NOT NULL,
    author TEXT NOT NULL,
    year_published INTEGER,
    genre TEXT
);
'''
cursor.execute(create_table_query)
books_data = [
    ("The Great Gatsby", "F. Scott Fitzgerald", 1925, "Fiction"),
    ("1984", "George Orwell", 1949, "Dystopian"),
    ("To Kill a Mockingbird", "Harper Lee", 1960, "Classic"),
]
cursor.executemany('INSERT INTO books (title, author, year_published, genre) VALUES (?, ?, ?, ?);', books_data)
cursor.execute('SELECT * FROM books;')
all_books = cursor.fetchall()
print("Все книги:", all_books)
cursor.execute('SELECT * FROM books WHERE year_published > 1950;')
books_after_1950 = cursor.fetchall()
print("Книги, опубликованные после 1950 года:", books_after_1950)
cursor.execute('SELECT * FROM books WHERE title LIKE "T%";')
books_starting_with_T = cursor.fetchall()
print("Книги, название которых начинается с 'T':", books_starting_with_T)
cursor.execute('SELECT * FROM books ORDER BY year_published ASC;')
sorted_books = cursor.fetchall()
print("Книги, отсортированные по году публикации:", sorted_books)
cursor.execute('SELECT * FROM books ORDER BY title LIMIT 2;')
first_two_books = cursor.fetchall()
print("Первые две книги, отсортированные по названию:", first_two_books)
cursor.execute('SELECT COUNT(*) FROM books;')
total_books = cursor.fetchone()[0]
print("Общее количество книг в таблице:", total_books)
cursor.execute('SELECT genre, COUNT(*) FROM books GROUP BY genre;')
books_by_genre = cursor.fetchall()
print("Количество книг для каждого жанра:", books_by_genre)
cursor.execute('UPDATE books SET year_published = 1948 WHERE title = "1984";')
cursor.execute('DELETE FROM books WHERE title = "The Great Gatsby";')
cursor.execute('CREATE INDEX IF NOT EXISTS idx_author ON books (author);')
cursor.execute("SELECT name FROM sqlite_master WHERE type='index' AND name='idx_author';")
index_exists = cursor.fetchone() is not None
print("Индекс для столбца author существует:", index_exists)
cursor.execute('''
CREATE TABLE IF NOT EXISTS logs (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    action TEXT,
    timestamp DATETIME DEFAULT CURRENT_TIMESTAMP
);
''')
cursor.execute('''
CREATE TRIGGER IF NOT EXISTS log_new_book AFTER INSERT ON books
BEGIN
    INSERT INTO logs (action) VALUES ('Добавлена новая книга: ' || NEW.title);
END;
''')
cursor.execute('INSERT INTO books (title, author, year_published, genre) VALUES (?, ?, ?, ?);',
               ("Brave New World", "Aldous Huxley", 1932, "Dystopian"))
try:
    conn.execute('BEGIN TRANSACTION;')
    cursor.execute('INSERT INTO books (title, author, year_published, genre) VALUES (?, ?, ?, ?);',
                   ("Fahrenheit 451", "Ray Bradbury", 1953, "Dystopian"))
    cursor.execute('UPDATE books SET year_published = 1950 WHERE title = "To Kill a Mockingbird";')
    conn.commit()
except Exception as e:
    print("Ошибка:", e)
    conn.rollback()
def calculate_book_age(year_published):
      current_year = 2023
      return current_year - year_published
conn.create_function("book_age", 1, calculate_book_age)
cursor.execute('SELECT title, book_age(year_published) AS age FROM books;')
books_with_age = cursor.fetchall()
print("Возраст книг:", books_with_age)
cursor.execute('CREATE VIEW IF NOT EXISTS modern_books AS SELECT * FROM books WHERE year_published > 1950;')
cursor.execute('SELECT * FROM modern_books;')
modern_books = cursor.fetchall()
print("Современные книги:", modern_books)
cursor.execute('DROP TABLE IF EXISTS books;')
cursor.execute("SELECT name FROM sqlite_master WHERE type='table' AND name='books';")
table_exists = cursor.fetchone() is not None
print("Таблица books существует после удаления:", table_exists)
cursor.execute('CREATE TABLE IF NOT EXISTS books (id INTEGER PRIMARY KEY AUTOINCREMENT, title TEXT NOT NULL, author TEXT NOT NULL, year_published INTEGER, genre TEXT);') # Восстанавливаем таблицу для индексации
cursor.executemany('INSERT INTO books (title, author, year_published, genre) VALUES (?, ?, ?, ?);', books_data) # Вставляем данные обратно
cursor.execute('CREATE INDEX IF NOT EXISTS idx_author_year ON books (author, year_published);')
if os.path.exists('library.db'):
    os.remove('library.db')
    print("База данных library.db удалена.")
else:
    print("База данных library.db не найдена.")
conn.close()

Все книги: [(1, 'The Great Gatsby', 'F. Scott Fitzgerald', 1925, 'Fiction'), (2, '1984', 'George Orwell', 1949, 'Dystopian'), (3, 'To Kill a Mockingbird', 'Harper Lee', 1960, 'Classic')]
Книги, опубликованные после 1950 года: [(3, 'To Kill a Mockingbird', 'Harper Lee', 1960, 'Classic')]
Книги, название которых начинается с 'T': [(1, 'The Great Gatsby', 'F. Scott Fitzgerald', 1925, 'Fiction'), (3, 'To Kill a Mockingbird', 'Harper Lee', 1960, 'Classic')]
Книги, отсортированные по году публикации: [(1, 'The Great Gatsby', 'F. Scott Fitzgerald', 1925, 'Fiction'), (2, '1984', 'George Orwell', 1949, 'Dystopian'), (3, 'To Kill a Mockingbird', 'Harper Lee', 1960, 'Classic')]
Первые две книги, отсортированные по названию: [(2, '1984', 'George Orwell', 1949, 'Dystopian'), (1, 'The Great Gatsby', 'F. Scott Fitzgerald', 1925, 'Fiction')]
Общее количество книг в таблице: 3
Количество книг для каждого жанра: [('Classic', 1), ('Dystopian', 1), ('Fiction', 1)]
Индекс для столбца author существует: Tru